In [ ]:
import sys
sys.path.append('../')
sys.path.append('C:\\Users\\olegk\\Documents\\unifloc\\')
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from datetime import date
from os import walk
import os

import uniflocpy.uTools.plotly_workflow as pw
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, plot

In [ ]:
def plot_corr_heatmap(df, filename_str):
    corrs = df.corr()
    figure = ff.create_annotated_heatmap(
       z=corrs.values,
       x=list(corrs.columns),
       y=list(corrs.index),
       annotation_text=corrs.round(2).values,
       showscale=True)
    figure.layout.update(
        margin=go.layout.Margin(
            l=400,
            r=50,
            b=100,
            t=250
        ))
    filename_str +='.html'
    plot(figure,filename=filename_str)
    
def initial_editing(data, wellname):
    if len(data.columns)==4:  
        del data[0]
    data.columns = [0, 1, 2]
    test_str = data[0][0]
    index = test_str.find(wellname)
    str_to_delete = test_str[:index] + wellname + '. '
    data[0] = data[0].str.replace(str_to_delete,"")
    data.index = data[1]
    del data[1]
    data.index = pd.to_datetime(data.index)
    return data

def extract_df_one_parametr_and_edit(data, list_of_params, number_of_param_in_list):
    extracted_df_one_param = data[data[0] == list_of_params[number_of_param_in_list]].copy()
    edited_df_one_param = extracted_df_one_param.rename(index=str, columns = {2: extracted_df_one_param[0][0]})
    del edited_df_one_param[0]
    return edited_df_one_param

def create_edited_df(data, parametrs_list):  # ускорить процесс
    init_one_parametr_df = extract_df_one_parametr_and_edit(data, parametrs_list, 0)
    result = init_one_parametr_df
    for i in range(1, len(parametrs_list)):
        new_one_parametr_df = extract_df_one_parametr_and_edit(data, parametrs_list, i)
        result = result.join([new_one_parametr_df], how = "outer", sort=True)
    return result

In [ ]:
current_path = os.getcwd()
path_to_data = current_path + "\\data\\"
dirnames_list = []
for (dirpath, dirnames, filenames) in walk(path_to_data):
    dirnames_list.extend(dirnames)
    break
dirnames_list

In [ ]:
files = dirnames_list
#files = ['1479', "1509"]
test_file = None

In [ ]:
for this_file_name in files[19:]: #file error 627 (17), 689(19), 1509(3),202(7) 
    print("Первичная обработка данных со СУ для скважины " + this_file_name)
    
    this_file_path = current_path + "\\data\\" + this_file_name +  "\\" + this_file_name + ".csv"
    test_file = pd.read_csv(this_file_path,sep=';', header=None)
    edited_file = initial_editing(test_file, this_file_name)
    big_list_parametrs = np.unique(edited_file[0])
    edited_file = create_edited_df(edited_file, big_list_parametrs)
    this_file_path_out = current_path + "\\data\\" + this_file_name +  "\\" + this_file_name + "_first_edit" +  ".csv"
    print("Сохранение обработанных данных в:")
    print(this_file_path_out)
    edited_file.to_csv(this_file_path_out)

In [ ]:
error_wells = ['627', '689', '1509', '202','1540','353', '252']

In [ ]:
well = '1354'

In [ ]:
this_file_path = current_path + "\\data\\" + well +  "\\" + well + "_first_edit" +  ".csv"

In [ ]:
test = pd.read_csv(this_file_path, index_col = '1')

In [ ]:
test.index = pd.to_datetime(test.index)

In [ ]:
test.head()

In [ ]:
for i in files:
    if i not in error_wells:
        print("Построение графиков для скважины " + i)
        well = i
        this_file_path = current_path + "\\data\\" + well +  "\\" + well + "_first_edit" +  ".csv"
        test = pd.read_csv(this_file_path, index_col = '1')
        test.index = pd.to_datetime(test.index)
        test_resampled = test.resample('3h').mean()

        test_resampled = test_resampled.dropna(subset = ['Объемный дебит жидкости'])

        all_traces = pw.create_traces_list_for_all_columms(test_resampled, chosen_mode = 'lines+markers')

        data = all_traces
        path_to_plot = current_path + "\\data\\" + well +  "\\" + \
                    'Исходные данные (first_edit) по 3 часа без интерполяции для ' + well +  '.html'
        print("Сохранение графика в:")
        print(path_to_plot)
        pw.plot_subplots(data, path_to_plot, True)
        #plot_corr_heatmap(test_resampled, "Корреляционная карта для данных по 3 часа без интерполяции")

In [ ]:
#test = pd.read_csv("edited_kobzar2.csv", index_col = "2")
#test.index = pd.to_datetime(test.index)
#
#test_resampled = test.resample('3h').mean()
#
#test_resampled = test_resampled.dropna(subset = [' Объемный дебит жидкости'])
#
#all_traces = pw.create_traces_list_for_all_columms(test_resampled, chosen_mode = 'lines')
#
#data = all_traces
#pw.plot_subplots(data, 'Исходные данные по 3 часа без интерполяции' + '.html', True)
#